In [ ]:
import os
TARGET = '5511'
CONESFILE1 = '.\\data\\20220601.ppdc-ases.txt'
CONESFILE2 = '.\\data\\20220801.ppdc-ases.txt'
PATHSFILE = '.\\data\\20220801.all-paths'

In [ ]:
# find target cone in each of the cone files

current_line = '0 0'
if os.path.isfile(CONESFILE1):
    cdata = open(CONESFILE1, 'r', encoding='utf-8')
    # print(type(cdata1))
    while current_line[0:current_line.index(' ')] != TARGET:
        current_line = cdata.readline()
        if len(current_line) == 0:
            print('didn\'t find it')
            break
    # print(current_line)
else:
    print('bad filename (cones file 1)')

tcone1 = current_line[(current_line.index(' ')+1):len(current_line)]

current_line = '0 0'
if os.path.isfile(CONESFILE2):
    cdata = open(CONESFILE2, 'r', encoding='utf-8')
    # print(type(cdata1))
    while current_line[0:current_line.index(' ')] != TARGET:
        current_line = cdata.readline()
        if len(current_line) == 0:
            print('didn\'t find it')
            break
    # print(current_line)
else:
    print('bad filename (cones file 2)')

tcone2 = current_line[(current_line.index(' ')+1):len(current_line)]

In [ ]:
# convert target cones into dicts for faster filtering of lost and gained ASes

tc1d = {}
for asn in tcone1.split(' '):
    tc1d[int(asn)] = True

tc2d = {}
for asn in tcone2.split(' '):
    tc2d[int(asn)] = True

lost_asns = []
gained_asns = []

for asn in tc1d:
    if not int(asn) in tc2d:
        lost_asns.append(int(asn))

for asn in tc2d:
    if not int(asn) in tc1d:
        gained_asns.append(int(asn))

In [ ]:
# analyzes paths file and finds path frequencies
# for LOST ASes

lost_paths = []
lost_asn_pcounts = {}
targfound = False
lostfound = False
curlost = []
for asn in lost_asns:
    lost_asn_pcounts[asn] = 0
progcounter = 0

if os.path.isfile(PATHSFILE):
    with open(PATHSFILE, 'r', encoding='utf-8') as paths:
        current_line = paths.readline()
        while len(current_line) != 0:
            curlost = []
            for asn in current_line.split(' ')[1].split('|'):
                if lost_asns.count(int(asn)) != 0:
                    lostfound = True
                    curlost.append(int(asn))
                if asn == TARGET:
                    targfound = True
            if targfound and lostfound:
                lost_paths.append(current_line.split(' ')[1])
                for asn in curlost:
                    lost_asn_pcounts[asn] += 1
        
            current_line = paths.readline()
            targfound = False
            lostfound = False
            progcounter += 1
            if progcounter % 1000000 == 0:
                print('Progress: ' + str(progcounter) + ' lines processed')
else:
    print('bad filename (paths file)')

In [ ]:
# analyzes paths file and finds path frequencies
# for GAINED ASes

gained_paths = []
gained_asn_pcounts = {}
targfound = False
gainedfound = False
curgained = []
for asn in gained_asns:
    gained_asn_pcounts[asn] = 0
progcounter = 0

if os.path.isfile(PATHSFILE):
    with open(PATHSFILE, 'r', encoding='utf-8') as paths:
        current_line = paths.readline()
        while len(current_line) != 0:
            curgained = []
            for asn in current_line.split(' ')[1].split('|'):
                if gained_asns.count(int(asn)) != 0:
                    gainedfound = True
                    curgained.append(int(asn))
                if asn == TARGET:
                    targfound = True
            if targfound and gainedfound:
                gained_paths.append(current_line.split(' ')[1])
                for asn in curgained:
                    gained_asn_pcounts[asn] += 1
        
            current_line = paths.readline()
            targfound = False
            gainedfound = False
            
            # I will have to make it a nice percentage at some point
            progcounter += 1
            if progcounter % 1000000 == 0:
                print('Progress: ' + str(progcounter) + ' lines processed')
            
else:
    print('bad filename (paths file)')

In [ ]:
# frequency counts
def getfcounts(asn_list, pcount_dict):
    fcounts = {}
    for asn in asn_list:
        if pcount_dict[asn] in fcounts:
            fcounts[pcount_dict[asn]] += 1
        else:
            fcounts[pcount_dict[asn]] = 1
    return fcounts
    
fcounts = getfcounts(gained_asns, gained_asn_pcounts)

# print frequency count
n = 0
for frequency, count in sorted(fcounts.items(), key=lambda f_c: f_c[1], reverse=True):
    print(frequency, count)
    n += 1
    if n > 20:
        break

In [ ]:
def savepaths(pathslist, s, date):
    OUTPUTNAME = '{}.{}_from_{}.txt'.format(date, s, TARGET)
    with open(OUTPUTNAME, 'a', encoding='utf-8') as output:
        for line in pathslist:
            output.write(line + '\n')

savepaths(gained_paths, 'gained', 20220801)

In [ ]:
# this one was just for fun

foundcount = 0
gainers2 = {}

for path in gained_paths:
    plist = path.split('|')
    for i in range(len(plist)):
        plist[i] = int(plist[i])
    foundcount = 0
    plist.reverse()
    for asn in plist:
        if not asn in gainers2:
            gainers2[asn] = 0
        if asn == int(TARGET):
            break
        gainers2[asn] += foundcount
        if gained_asns.count(asn) > 0:
            foundcount += 1
        else:
            foundcount += -99